In [1]:
import requests
import pandas as pd
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from itertools import chain

In [2]:
# Количество страниц с объявлениями для каждой марки машины
def pages(link):
    driver = webdriver.Chrome()
    driver.get(link)
    page = BeautifulSoup(driver.page_source, 'html.parser')
    pages = page.find_all(
        'span', class_='ControlGroup ControlGroup_responsive_no ControlGroup_size_s ListingPagination-module__pages')
    if pages == []:
        total_page_count = 1
    else:
        total_page_count = int(page.find_all(
            'span', class_='Button__text')[-4].text)
    driver.close()
    time.sleep(0.3)
    return total_page_count

# Разбиение линков на страницы для каждой модели
def ads_pages(link):
    pages_for_car = []
    pages_cnt = pages(link)
    if pages_cnt == 1:
        pages_for_car.append(link)
    else:
        for i in range(1, pages_cnt+1):
            link_page = link+'?page='+str(i)
            pages_for_car.append(link_page)
    return pages_for_car

# Получаем линки для каждого объявления
def get_ad_url(link):
    time.sleep(0.2)
    res = requests.get(link)
    page = BeautifulSoup(res.text, 'html.parser')
    ad_links = page.find_all('a', class_='Link ListingItemTitle-module__link')
    car_links = list(map(lambda x: x.get('href'), ad_links))

    return car_links


def car_data(link):
    try:
        time.sleep(0.1)
        res = requests.get(link)    
        res.encoding ='utf8'     #Это необходимо, т.к. в тексте много кириллицы
        page = BeautifulSoup(res.text, 'html.parser')
        car_dict = {}

        car_dict['car_url'] = link
        car_dict['parsing_unixtime'] = int(time.time())
        #Начинаем вынимать из кода необходимую информацию, в формате словарей
        # в позициях script ищем необходимые вхождения 
        for script in page.find_all("script"):
            if 'complectation":{"id"' in str(script):
                a = str(script)  # присваиваем a содержимое скрипта как строку
            # в a  ищем 'complectation":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
                car_dict['complectation_dict'] = re.search(r'complectation":{"id.*?}', a)[0][15:]
            if 'equipment":{' in str(script):
                a = str(script)  
            # в a  ищем 'equipment":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
                car_dict['equipment_dict'] = re.search(r'equipment":{.*?}', a)[0][11:]
            if '{"mileage":' in str(script):
                a = str(script)  
            # в a  ищем '{"mileage":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
                car_dict['mileage'] = re.search(r'"mileage":\d*', a)[0][10:]
            if '"model_info":' in str(script):
                a = str(script)  
                car_dict['model_info'] = re.search(r'"model_info":{.*?}', a)[0][13:]
                car_dict['model_name'] = re.search(r'model_info":{"code":".*?"', a)[0][20:].strip('"')
            if 'super_gen":{' in str(script):
                a = str(script)  
                car_dict['super_gen'] = re.search(r'tech_param":{.*?}', a)[0][12:] 
            if 'vendor":"' in str(script):
                a = str(script)  
                car_dict['vendor'] = re.search(r'vendor":".*?"', a)[0][9:].strip('"')

        # По соответствующим тэгам вынимаем  в словарь необходимую информацию
        for tag in page.find_all('div'):
            if tag.get("title") == "Идентификатор объявления":
                car_dict['sell_id'] = re.search(r'\d+', tag.text)[0]
        
        img_l = page.find('img', class_='ImageGalleryDesktop__image')
        car_dict['image'] = img_l.get('src')

        dict1 = eval(str(page.find("script"))[35:-9])
        car_dict['bodyType'] = dict1['bodyType']
        car_dict['brand'] = dict1['brand']
        car_dict['color'] = dict1['color']
        car_dict['description'] = dict1['description']
        car_dict['engineDisplacement'] = dict1['vehicleEngine']['engineDisplacement']
        car_dict['enginePower'] = dict1['vehicleEngine']['enginePower']
        car_dict['fuelType'] = dict1['fuelType']
        car_dict['modelDate'] = dict1['modelDate']
        car_dict['name'] = dict1['name']
        car_dict['numberOfDoors'] = dict1['numberOfDoors']
        car_dict['price'] = dict1['offers']['price']
        car_dict['priceCurrency'] = dict1['offers']['priceCurrency']
        car_dict['productionDate'] = dict1['productionDate']
        car_dict['vehicleConfiguration'] = dict1['vehicleConfiguration']
        car_dict['vehicleTransmission'] = dict1['vehicleTransmission']
        
        span_CardInfoRow__cell = page.find_all('span', {'class': 'CardInfoRow__cell'})
        for i,tag in enumerate (span_CardInfoRow__cell):
            if tag.text == "Владельцы":
                car_dict['Владельцы'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел
            if tag.text == "Владение":
                car_dict['Владение'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') 
            if tag.text == "ПТС":
                car_dict['ПТС'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ')     
            if tag.text == "Привод":
                car_dict['Привод'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ')  
            if tag.text == "Руль":
                car_dict['Руль'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') 
            if tag.text == "Состояние":
                car_dict['Состояние'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ')  
            if tag.text == "Таможня":
                car_dict['Таможня'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') 
        return car_dict
    except:
        pass

In [4]:
driver = webdriver.Chrome(r"C:\Python\chromedriver.exe")
driver.get('https://auto.ru/moskovskaya_oblast')

# Кликаем на ссылку, чтобы раскрыть список всех марок
driver.find_element_by_class_name('IndexMarks__show-all').click()

In [7]:
# Ставим небольшую задержку, т.к. сайту нужно время, чтобы открыть ссылку
time.sleep(2)

# На странице изменился код, теперь можно получить полный список марок с объявлениями
soup = BeautifulSoup(driver.page_source, 'html.parser')
marks_to_sale= soup.find('div', class_='IndexMarks__marks-with-counts')
driver.close()
marks_to_sale

<div class="IndexMarks__marks-with-counts"><div class="IndexMarks__col"><a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/ac/all/"><div class="IndexMarks__item-name">AC</div><div class="IndexMarks__item-count">1</div></a><a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/amc/all/"><div class="IndexMarks__item-name">AMC</div><div class="IndexMarks__item-count">4</div></a><a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/acura/all/"><div class="IndexMarks__item-name">Acura</div><div class="IndexMarks__item-count">45</div></a><a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/adler/all/"><div class="IndexMarks__item-name">Adler</div><div class="IndexMarks__item-count">1</div></a><a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/alfa_romeo/all/"><div class="IndexMarks__item-name">Alfa Romeo</div><div class="IndexMarks__item-count">41</div></a><a class="IndexMarks__item" hre

In [8]:
#Пока набор данных получен в выде списка из пяти записей
len(marks_to_sale)

5

In [9]:
# Вытащим записи с ссылками
marks = list(map(lambda x: x.find_all('a', class_='IndexMarks__item'), marks_to_sale))
marks

[[<a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/ac/all/"><div class="IndexMarks__item-name">AC</div><div class="IndexMarks__item-count">1</div></a>,
  <a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/amc/all/"><div class="IndexMarks__item-name">AMC</div><div class="IndexMarks__item-count">4</div></a>,
  <a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/acura/all/"><div class="IndexMarks__item-name">Acura</div><div class="IndexMarks__item-count">45</div></a>,
  <a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/adler/all/"><div class="IndexMarks__item-name">Adler</div><div class="IndexMarks__item-count">1</div></a>,
  <a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/alfa_romeo/all/"><div class="IndexMarks__item-name">Alfa Romeo</div><div class="IndexMarks__item-count">41</div></a>,
  <a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/alpina/

In [10]:
#Пока данные всё ещё в виде списка перечней
print(len(marks))
marks = list(chain(*marks))
print(len(marks))

5
132


In [11]:
#Составим список из ссылок. При этом заменим  категорию "все" на "подержанные"
ref = []
for m in marks:
    r = m.get('href')
    r = r.replace("/all/", "/used/")
    ref.append(r)
len(ref)

132

In [12]:
#Посмотрим, что же получилось
print(type(ref[1]), ref[1])

<class 'str'> https://auto.ru/moskovskaya_oblast/cars/amc/used/


In [17]:
ref_pages = []
ref_pages = Parallel(n_jobs=3)(delayed(ads_pages)(page_link) for page_link in tqdm(ref))

In [18]:
ref_pages_fin = list(chain(*ref_pages))
#Сохраним список ссылок на страницы в файл "про запас"
pd.Series(ref_pages_fin).to_csv('links_page.csv', index=False)

In [22]:
ref_pages = list(pd.read_csv('links_page.csv').values.flatten())

Получим список ссылок на все объявления о продаже подержанных машин

In [ ]:
ads = []
ads = Parallel(n_jobs=2)(delayed(get_ad_url)(page_link) for page_link in tqdm(ref_pages))

In [ ]:
#Преобразуем список, посмотрим, что получилось на случайно выбранном примере
ads_list = list(chain(*ads))
print(len(ads_list), ads_list[115])

In [ ]:
#Также сохраним этот список в файл
pd.Series(ads_list).to_csv('ads_links.csv', index=False)

In [3]:
#Извлечём список из файла
ads_list = list(pd.read_csv('ads_links.csv').values.flatten())

In [5]:
ads_list

53166

In [ ]:
car_data_list = []
for ads in tqdm(ads_list):
    try:
        car_data_l = car_data(ads)
        car_data_list.append(car_data_l)
    except:
        pass

In [ ]:
print(len(car_data_list),type(car_data_list[1]), car_data_list[11111])

In [3]:
data = pd.DataFrame()
for car in car_data_list:
    try:
        data = data.append(car, ignore_index=True)
    except:
        pass
data.to_csv('car_auto_ru_train.csv', encoding = 'utf-8', index=False)

NameError: name 'car_data_list' is not defined

In [61]:
df = pd.read_csv('car_auto_ru_info_train.csv')

In [65]:
df.car_url.iloc[33000]

'https://auto.ru/cars/used/sale/mitsubishi/mirage/1102873984-e407655d/'

In [7]:
# Вывод этой таблицы был осуществлен еще до последних изменений в функции car_data, в последнем варианте
# парсятся все теже признаки, что и в предоставленной тестовой выборке.
# Train в конечном варианте парсился в итоге со стационарного компьютера ночью.
df.T

,0,1,2,3,4,5,6,7,8,9,...,53156,53157,53158,53159,53160,53161,53162,53163,53164,53165
car_url,https://auto.ru/cars/used/sale/ac/cobra/108956...,https://auto.ru/cars/used/sale/amc/eagle/11017...,https://auto.ru/cars/used/sale/amc/eagle/10961...,https://auto.ru/cars/used/sale/amc/eagle/10962...,https://auto.ru/cars/used/sale/amc/eagle/10867...,https://auto.ru/cars/used/sale/acura/mdx/11027...,https://auto.ru/cars/used/sale/acura/tlx/11022...,https://auto.ru/cars/used/sale/acura/mdx/11028...,https://auto.ru/cars/used/sale/acura/rdx/11028...,https://auto.ru/cars/used/sale/acura/mdx/11029...,...,https://auto.ru/cars/used/sale/uaz/hunter/1102...,https://auto.ru/cars/used/sale/uaz/3151/110224...,https://auto.ru/cars/used/sale/uaz/patriot/110...,https://auto.ru/cars/used/sale/uaz/469/1102933...,https://auto.ru/cars/used/sale/uaz/hunter/1102...,https://auto.ru/cars/used/sale/uaz/patriot/102...,https://auto.ru/cars/used/sale/uaz/patriot/110...,https://auto.ru/cars/used/sale/uaz/3151/110292...,https://auto.ru/cars/used/sale/uaz/patriot/110...,https://auto.ru/cars/used/sale/uaz/patriot/110...
complectation_dict,"{""id"":""0""}","{""id"":""0""}","{""id"":""0""}","{""id"":""0""}","{""id"":""0""}","{""id"":""20079925"",""name"":""Advance"",""available_o...","{""id"":""0""}","{""id"":""0""}","{""id"":""0""}","{""id"":""0""}",...,"{""id"":""0""}","{""id"":""0""}","{""id"":""0""}","{""id"":""0""}","{""id"":""0""}","{""id"":""0""}","{""id"":""10417387"",""name"":""Classic"",""available_o...","{""id"":""0""}","{""id"":""0""}","{""id"":""0""}"
equipment_dict,"{""alloy-wheel-disks"":true,""14-inch-wheels"":tru...",{},{},"{""electro-window-back"":true,""roof-rails"":true,...",{},"{""esp"":true,""usb"":true,""windcleaner-heat"":true...",{},"{""cruise-control"":true,""asr"":true,""esp"":true,""...","{""cruise-control"":true,""engine-proof"":true,""as...","{""cruise-control"":true,""start-stop-function"":t...",...,{},{},{},{},{},{},"{""alloy-wheel-disks"":true,""paint-metallic"":tru...","{""alloy-wheel-disks"":true,""tinted-glass"":true,...","{""tinted-glass"":true,""aux"":true,""usb"":true,""sp...","{""alloy-wheel-disks"":true,""ptf"":true,""seats-5""..."
mileage,13500,60000,140000,70000,140000,103000,28295,190000,135592,105735,...,139000,33000,64800,52000,43000,88200,21600,81000,98000,114000
model_info,"{""code"":""COBRA"",""name"":""Cobra"",""ru_name"":""Кобр...","{""code"":""EAGLE"",""name"":""Eagle"",""ru_name"":""Игл""...","{""code"":""EAGLE"",""name"":""Eagle"",""ru_name"":""Игл""...","{""code"":""EAGLE"",""name"":""Eagle"",""ru_name"":""Игл""...","{""code"":""EAGLE"",""name"":""Eagle"",""ru_name"":""Игл""...","{""code"":""MDX"",""name"":""MDX"",""ru_name"":""мдх"",""mo...","{""code"":""TLX"",""name"":""TLX"",""ru_name"":""тлх"",""mo...","{""code"":""MDX"",""name"":""MDX"",""ru_name"":""мдх"",""mo...","{""code"":""RDX"",""name"":""RDX"",""ru_name"":""РДХ"",""mo...","{""code"":""MDX"",""name"":""MDX"",""ru_name"":""мдх"",""mo...",...,"{""code"":""HUNTER"",""name"":""Hunter"",""ru_name"":""Ха...","{""code"":""3151"",""name"":""3151"",""ru_name"":""3151"",...","{""code"":""PATRIOT"",""name"":""Patriot"",""ru_name"":""...","{""code"":""469"",""name"":""469"",""ru_name"":""469"",""mo...","{""code"":""HUNTER"",""name"":""Hunter"",""ru_name"":""Ха...","{""code"":""PATRIOT"",""name"":""Patriot"",""ru_name"":""...","{""code"":""PATRIOT"",""name"":""Patriot"",""ru_name"":""...","{""code"":""3151"",""name"":""3151"",""ru_name"":""3151"",...","{""code"":""PATRIOT"",""name"":""Patriot"",""ru_name"":""...","{""code"":""PATRIOT"",""name"":""Patriot"",""ru_name"":""..."
model_name,COBRA,EAGLE,EAGLE,EAGLE,EAGLE,MDX,TLX,MDX,RDX,MDX,...,HUNTER,3151,PATRIOT,469,HUNTER,PATRIOT,PATRIOT,3151,PATRIOT,PATRIOT
name,4.9 MT (326 л.с.),2.5 MT (85 л.с.) 4WD,4.2 MT (114 л.с.) 4WD,4.2 AT (112 л.с.) 4WD,4.2 MT (114 л.с.) 4WD,3.5 AT (290 л.с.) 4WD,2.4 AMT (208 л.с.),3.5 AT (240 л.с.) 4WD,2.3 AT (240 л.с.) 4WD,3.5 AT (290 л.с.) 4WD,...,315143 2.4d MT (86 л.с.) 4WD,3151 2.5 MT (76 

In [3]:
# Пример рабочего парсинга
car_data('https://auto.ru/cars/used/sale/ford/explorer/1102718617-ded047ab/')

{'car_url': 'https://auto.ru/cars/used/sale/ford/explorer/1102718617-ded047ab/',
 'parsing_unixtime': 1619506059,
 'complectation_dict': '{"id":"2460464","name":"LTD","available_options":["cruise-control","airbag-passenger","lock","electro-mirrors","mirrors-heat","leather","computer","seat-transformation","wheel-power","airbag-side","abs","climate-control-1","esp","audiopreparation","electro-window-back","hatch","condition","music-super","airbag-driver","electro-window-front","passenger-seat-electric","paint-metallic","audiosystem-cd","migration-flag","front-seats-heat","wheel-configuration2","wheel-configuration1","immo"]}',
 'equipment_dict': '{"cruise-control":true,"asr":true,"tinted-glass":true,"esp":true,"usb":true,"sport-seats":true,"multi-wheel":true,"spare-wheel":true,"xenon":true,"audiosystem-tv":true,"heated-wash-system":true,"ashtray-and-cigarette-lighter":true,"airbag-passenger":true,"navigation":true,"bas":true,"isofix-front":true,"lock":true,"door-sill-panel":true,"drl":t